# Sentiment Analysis Model

## Model Architecture
- **Base Model**: [DistilBERT](https://huggingface.co/distilbert-base-uncased) (`distilbert-base-uncased`), a lightweight transformer-based model.

## Fine-Tuning
- **Dataset**: IMDB movie reviews (25,000 samples for training, preprocessed with tokenization, truncation, and padding).
- **Task**: Binary classification (`LABEL_0`: Negative, `LABEL_1`: Positive).
- **Hyperparameters**:
  - Learning rate: `2e-5`
  - Batch size: `8`
  - Number of epochs: `3`
  - Weight decay: `0.01`

## Training Details
- **Framework**: Hugging Face `Trainer`.
- **Compute Resources**: A100 GPU, High RAM.
- **Runtime**: ~20 minutes.

## Inference Pipeline
- **Library**: Hugging Face `pipeline` for text classification.
- **Label Mapping**:
  - `LABEL_0`: 😡 Negative
  - `LABEL_1`: 😊 Positive
- **Output**: Predicted sentiment and confidence score (0-100%).

## Deployment
- **Interface**: [Gradio](https://gradio.app) for web-based predictions.
- **Features**:
  - Text input box for user reviews.
  - Outputs sentiment with emoji and confidence percentage.

## Saved Model
- Path: `/content/drive/My Drive/movie_sentiment_model`

## Example Usage
### Input
```python
sample_text = "This movie was absolutely fantastic!"
```

### Output
```python
output = {
  "label": "😊 Positive",
  "confidence": "99.82%"
}
```



## Train LLM

In [2]:
# Install required libraries
!pip install transformers datasets

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the pre-trained tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load a dataset (e.g., IMDB for sentiment analysis)
dataset = load_dataset("imdb")

# Preprocess dataset for fine-tuning
def preprocess_data(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

# Convert to Hugging Face Dataset
train_dataset = dataset['train'].map(preprocess_data, batched=True)
test_dataset = dataset['test'].map(preprocess_data, batched=True)

# Load a pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Import additional libraries
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

# Define a custom compute_metrics function for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": accuracy, "f1": f1}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    push_to_hub=False,
)

# Create Trainer with compute_metrics added
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Include metrics
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print(metrics)  # Output includes accuracy and F1-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.243100,0.283808,0.914520,0.914371
2,0.204700,0.279043,0.930840,0.930839
3,0.096700,0.328533,0.932200,0.932199


{'eval_loss': 0.32853344082832336, 'eval_accuracy': 0.9322, 'eval_f1': 0.9321988933774582, 'eval_runtime': 97.8598, 'eval_samples_per_second': 255.468, 'eval_steps_per_second': 31.933, 'epoch': 3.0}


In [3]:
# Save Locally
model.save_pretrained("./movie_sentiment_model")
tokenizer.save_pretrained("./movie_sentiment_model")
# Save to drive
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained("/content/drive/My Drive/movie_sentiment_model")
tokenizer.save_pretrained("/content/drive/My Drive/movie_sentiment_model")


Mounted at /content/drive


('/content/drive/My Drive/movie_sentiment_model/tokenizer_config.json',
 '/content/drive/My Drive/movie_sentiment_model/special_tokens_map.json',
 '/content/drive/My Drive/movie_sentiment_model/vocab.txt',
 '/content/drive/My Drive/movie_sentiment_model/added_tokens.json',
 '/content/drive/My Drive/movie_sentiment_model/tokenizer.json')

In [4]:
# Load the saved model and tokenizer
from transformers import pipeline
label_map = {"LABEL_0": "Negative", "LABEL_1": "Positive"}

sentiment_analyzer = pipeline("text-classification", model="./movie_sentiment_model")

# Predict sentiment 1
sample_text = "This movie was absolutely fantastic!"

result = sentiment_analyzer(sample_text)
for res in result:
    res["label"] = label_map[res["label"]]
print("\nsample_text: ",sample_text)
print(result)

# Predict sentiment 2
sample_text = "This movie was so bad!"

result = sentiment_analyzer(sample_text)
for res in result:
    res["label"] = label_map[res["label"]]
print("\nsample_text: ",sample_text)
print(result)


Device set to use cuda:0



sample_text:  This movie was absolutely fantastic!
[{'label': 'Positive', 'score': 0.9981303811073303}]

sample_text:  This movie was so bad!
[{'label': 'Negative', 'score': 0.9985581040382385}]


## Load the tuned LLM from drive

In [5]:
!pip install gradio

In [6]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the path to the saved model and tokenizer in Google Drive
model_path = "/content/drive/My Drive/movie_sentiment_model"

# Import necessary libraries
from transformers import pipeline

# Define the label mapping
label_map = {"LABEL_0": "Negative", "LABEL_1": "Positive"}

# Load the saved model and tokenizer from Google Drive
sentiment_analyzer = pipeline("text-classification", model=model_path, tokenizer=model_path)

# Predict sentiment 1
sample_text = "This movie was absolutely fantastic!"
result = sentiment_analyzer(sample_text)
for res in result:
    res["label"] = label_map[res["label"]]
print("\nsample_text: ", sample_text)
print(result)

# Predict sentiment 2
sample_text = "This movie was so bad!"
result = sentiment_analyzer(sample_text)
for res in result:
    res["label"] = label_map[res["label"]]
print("\nsample_text: ", sample_text)
print(result)


Device set to use cuda:0


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

sample_text:  This movie was absolutely fantastic!
[{'label': 'Positive', 'score': 0.9981303811073303}]

sample_text:  This movie was so bad!
[{'label': 'Negative', 'score': 0.9985581040382385}]


## Host the LLM

In [7]:
from transformers import pipeline
import gradio as gr

# Set the path to your saved fine-tuned model and tokenizer
model_path = "/content/drive/My Drive/movie_sentiment_model"

# Load the fine-tuned model and tokenizer from the local directory
sentiment_analyzer = pipeline("text-classification", model=model_path, tokenizer=model_path)

# Define label mapping with emojis
label_mapping = {
    "LABEL_0": "😡 Negative",
    "LABEL_1": "😊 Positive"
}

# Define prediction function
def predict_sentiment(text):
    results = sentiment_analyzer(text)
    label = label_mapping[results[0]['label']]  # Convert LABEL_0 or LABEL_1 to human-readable text with emojis
    confidence = results[0]['score'] * 100  # Convert confidence to percentage
    return f"{label}\nConfidence: {confidence:.2f}%"

# Create Gradio interface
interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=2, placeholder="Enter a movie review here..."),
    outputs="text",
    title="🎬 Sentiment Analysis",
    description="Analyze the positive-negative sentiment of any movie review with emojis! 😊😡",
    theme="default"
)

print("\nLABEL_0: 😡 Negative, LABEL_1: 😊 Positive\n")
# Launch the Gradio app
interface.launch(share=True)


Device set to use cuda:0



LABEL_0: 😡 Negative, LABEL_1: 😊 Positive

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ba196a484389c07354.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
